**Simple Quick EDA**
* One id have multiple encodings
* 3 different types of cell types. Out of these 3, shsy5y is dominant in dataset.
* All the images have fixed size.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from glob import glob

In [ ]:
live_path='../input/sartorius-cell-instance-segmentation'
train_path='../input/sartorius-cell-instance-segmentation/train'
test_path='../input/sartorius-cell-instance-segmentation/test'

In [ ]:
train_df=pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
sample_sub=pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
train_df.head()

In [ ]:
#Shape and length dataset
print('Train df shape: ',train_df.shape)
print('Total Unique Ids: ',train_df['id'].nunique())

In [ ]:
#Let's check if all the images have same Dimensions
print('Total Unique Widths: ',train_df['width'].nunique())
print('Total Unique Heights: ',train_df['height'].nunique())

In [ ]:
#Cell types
print('Types of cells: ',train_df['cell_type'].unique())

#Distribution of cell types
sns.displot(train_df['cell_type'])

In [ ]:
#Now comes the main thing
#let's take a look at some images and some masks then will take a loot at both

In [ ]:
unique_ids=train_df['id'].unique()

r,c=2,2
fig=plt.figure(figsize=(12,12))
for i in range(1,r*c+1):
    img=cv2.imread(os.path.join(train_path,unique_ids[i]+'.png'))
    fig.add_subplot(r,c,i)
    plt.imshow(img)
plt.show()

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
#Lets plot some images along with masks
r,c=3,3
fig=plt.figure(figsize=(18,18))

for i in range(1,r*c+1):
    img=cv2.imread(os.path.join(train_path,unique_ids[i]+'.png'))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    fig.add_subplot(r,c,i)
    
    #Get annot
    annots_=train_df['annotation'][train_df['id']==unique_ids[i]]
    mask=np.zeros((502,704))
    for ann in annots_.tolist():
        mask+=rle_decode(ann,(502, 704))
    
    mask=np.clip(mask,0,1)
    
    #Get cell type
    ctype=train_df['cell_type'][train_df['id']==unique_ids[i]].values[0]
    
    plt.imshow(img)
    plt.imshow(mask,alpha=0.2,cmap='gray')
    plt.title('Cell Type: '+ctype)
plt.show()

**What's next?**
* Create a Classifier to classify images into 3 cell types
* Create segmentation Model.